In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from lightgbm import LGBMRegressor, early_stopping
import warnings
warnings.filterwarnings('ignore')

In [2]:
asdasd = pd.read_csv('/root/code/좌표대체후v3.csv')

In [3]:
asdasd.columns

Index(['시군구_원본', '번지', '본번', '부번', '아파트명', '전용면적(㎡)', '계약년월', '계약일', '층',
       '건축년도', '도로명', '해제사유발생일', '등기신청일자', '거래유형', '중개사소재지',
       'k-단지분류(아파트,주상복합등등)', 'k-전화번호', 'k-팩스번호', '단지소개기존clob', 'k-세대타입(분양형태)',
       'k-관리방식', 'k-복도유형', 'k-난방방식', 'k-전체동수', 'k-전체세대수', 'k-건설사(시공사)',
       'k-시행사', 'k-사용검사일-사용승인일', 'k-연면적', 'k-주거전용면적', 'k-관리비부과면적',
       'k-전용면적별세대현황(60㎡이하)', 'k-전용면적별세대현황(60㎡~85㎡이하)', 'k-85㎡~135㎡이하',
       'k-135㎡초과', 'k-홈페이지', 'k-등록일자', 'k-수정일자', '고용보험관리번호', '경비비관리형태',
       '세대전기계약방법', '청소비관리형태', '건축면적', '주차대수', '기타/의무/임대/임의=1/2/3/4', '단지승인일',
       '사용허가여부', '관리비 업로드', '좌표X_원본', '좌표Y_원본', '단지신청일', 'target'],
      dtype='object')

In [6]:
# -------------------------
# 1. 라이브러리 및 설정
# -------------------------
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from lightgbm import LGBMRegressor, early_stopping
import warnings
warnings.filterwarnings('ignore')

# -------------------------
# 2. 데이터 로드
# -------------------------
train_path = '/root/train.csv'
test_path  = '/root/test.csv'
subway_path= '/root/subway_feature.csv'

dt_train = pd.read_csv(train_path)
dt_test  = pd.read_csv(test_path)
dt_subway= pd.read_csv(subway_path)


dt_train['is_test'] = 0
dt_test['is_test']  = 1
dt_all = pd.concat([dt_train, dt_test])

selected_columns = ['시군구','아파트명','전용면적(㎡)','계약년월','건축년도','도로명',
                    'k-세대타입(분양형태)','k-건설사(시공사)','좌표X','좌표Y',
                    'target','is_test']
dt_all = dt_all[selected_columns]
dt_all.columns = dt_all.columns.str.replace('k-', '', regex=False)
dt_all = dt_all.rename(columns={'좌표X':'경도','좌표Y':'위도'})

# 구/동 추출
dt_all['구'] = dt_all['시군구'].str.extract(r'([가-힣]+구)')[0]
dt_all['동'] = dt_all['시군구'].str.extract(r'([가-힣]+동)')[0]
dt_all = dt_all.drop('시군구', axis=1)

# 계약년도/월, 경과년수 계산
dt_all['계약년도'] = dt_all['계약년월'] // 100
dt_all['계약월'] = dt_all['계약년월'] % 100
dt_all['계약경과년수'] = dt_all['계약년도'] - 2007
dt_all['계약경과년수_로그'] = np.log1p(dt_all['계약경과년수'])
dt_all['계약경과년수_제곱'] = dt_all['계약경과년수'] ** 2
dt_all = dt_all.drop(['계약년월','계약월'], axis=1)

# 건물 연령 관련
dt_all['매매당시_건물연령'] = dt_all['계약년도'] - dt_all['건축년도']
dt_all['매매당시_신축여부'] = (dt_all['매매당시_건물연령']<=3).astype(int)
dt_all['매매당시_재건축대상'] = (dt_all['매매당시_건물연령']>=30).astype(int)

# 전용면적 변환
dt_all['전용면적_로그'] = np.log1p(dt_all['전용면적(㎡)'])
dt_all['전용면적_제곱'] = dt_all['전용면적(㎡)']**2

# 좌표X, 좌표Y 불러오기 
dt_all['위도'] = asdasd['좌표Y_원본']
dt_all['경도'] = asdasd['좌표X_원본']


# 결측값 처리
categorical_columns = dt_all.select_dtypes(include=['object']).columns.tolist()
numeric_columns     = dt_all.select_dtypes(include=['float64','int64']).columns.tolist()
dt_all[categorical_columns] = dt_all[categorical_columns].fillna('NULL')
dt_all[numeric_columns]     = dt_all[numeric_columns].fillna(0)

# -------------------------
# 4. Label Encoding
# -------------------------
for col in categorical_columns:
    le = LabelEncoder()
    dt_all[col] = le.fit_transform(dt_all[col].astype(str))

# -------------------------
# 5. train/test 분리
# -------------------------
dt_train = dt_all.query('is_test==0').drop(['is_test'], axis=1)
dt_test  = dt_all.query('is_test==1').drop(['is_test','target'], axis=1)

# -------------------------
# 6. Growth bin 생성
# -------------------------
trend_df = dt_train.groupby('계약경과년수')['target'].mean().reset_index()
trend_df = trend_df.sort_values('계약경과년수')
n_bins = 6
total_years = len(trend_df)
bin_size = total_years // n_bins

year_bins = []
for i in range(n_bins):
    start_idx = i * bin_size
    end_idx = (i+1) * bin_size if i < n_bins-1 else total_years
    years_in_bin = trend_df['계약경과년수'].iloc[start_idx:end_idx].tolist()
    year_bins.append(years_in_bin)

contract_to_bin = {}
for i, bin_years in enumerate(year_bins):
    for year in bin_years:
        contract_to_bin[year] = i

dt_train['growth_bin'] = dt_train['계약경과년수'].map(contract_to_bin)
dt_test['growth_bin']  = dt_test['계약경과년수'].map(contract_to_bin)

# -------------------------
# 7. Policy feature
# -------------------------
dt_train['policy_mj'] = ((dt_train['계약경과년수'] >= 12) & (dt_train['계약경과년수'] <= 15)).astype(int)
dt_test['policy_mj']  = ((dt_test['계약경과년수'] >= 12) & (dt_test['계약경과년수'] <= 15)).astype(int)

# -------------------------
# 8. Sample weight 설정
# -------------------------
bin_weights = {i:1.0 for i in range(n_bins)}
bin_weights[4] = 0.8
dt_train['sample_weight'] = dt_train['growth_bin'].map(bin_weights)

# -------------------------
# 9. X_train / y_train 생성
# -------------------------
X_train = dt_train.drop('target', axis=1)
y_train = dt_train['target']

# -------------------------
# 10. train/validation split
# -------------------------
Xy = X_train.copy()
Xy['target'] = y_train
Xy = Xy.sort_values('계약경과년수', ascending=False)

split_idx = int(len(Xy)*0.8)
train = Xy.iloc[:split_idx]
val   = Xy.iloc[split_idx:]

X_tr, y_tr = train.drop('target', axis=1), train['target']
X_val, y_val = val.drop('target', axis=1), val['target']
weights_tr = train['sample_weight']
weights_val = val['sample_weight']

# -------------------------
# 11. RandomForest 학습 및 예측
# -------------------------
rf = RandomForestRegressor(
    n_estimators=400, max_depth=35, min_samples_split=5, min_samples_leaf=1,
    random_state=42, n_jobs=-1
)
rf.fit(X_tr, y_tr, sample_weight=weights_tr)
pred_rf_val = rf.predict(X_val)
rmse_rf = np.sqrt(mean_squared_error(y_val, pred_rf_val))
print(f"RandomForest RMSE (validation): {rmse_rf:.2f}")

# 테스트 데이터 예측
X_test_rf = dt_test[X_train.columns]
pred_rf_test = rf.predict(X_test_rf)

# RandomForest 결과 저장
sample = pd.read_csv("/root/sample_submission.csv")
submission_rf = sample.copy()
submission_rf['target'] = pred_rf_test.round().astype(int)
submission_rf.to_csv("submission_rf_v3.csv", index=False)
print("✅ submission_rf_v3.csv 저장 완료!")

# -------------------------
# 12. LightGBM 학습
# -------------------------
best_params = {
    'learning_rate': 0.065,
    'num_leaves': 136,
    'min_data_in_leaf': 36,
    'feature_fraction': 0.626,
    'bagging_fraction': 0.936,
    'bagging_freq': 3,
    'n_estimators': 5000,
    'random_state': 42,
    'n_jobs': -1
}

lgbm = LGBMRegressor(**best_params)
lgbm.fit(
    X_tr, np.log1p(y_tr),
    eval_set=[(X_val, np.log1p(y_val))],
    eval_metric='rmse',
    callbacks=[early_stopping(200)],
    sample_weight=weights_tr
)

# Validation 예측
pred_log_val = lgbm.predict(X_val, num_iteration=lgbm.best_iteration_)
pred_lgbm_val = np.expm1(pred_log_val)
rmse_lgbm = np.sqrt(mean_squared_error(y_val, pred_lgbm_val))
print(f"LightGBM RMSE (validation): {rmse_lgbm:.2f}")

# -------------------------
# 13. LightGBM 최종 학습 & 테스트 예측
# -------------------------
lgbm.fit(X_train, np.log1p(y_train))
test_pred_log = lgbm.predict(X_test_rf, num_iteration=lgbm.best_iteration_)
test_pred = np.expm1(test_pred_log).round().astype(int)

submission_lgbm = sample.copy()
submission_lgbm['target'] = test_pred
submission_lgbm.to_csv("submission_v3.csv", index=False)
print("✅ submission_v3.csv 저장 완료!")

# -------------------------
# 14. 최종 사용 feature 확인
# -------------------------
print("✅ 최종 학습에 사용된 feature 컬럼:")
print(X_train.columns.tolist())


KeyboardInterrupt: 